# IS 477 Midterm Notebook
### Project: Mapping and Modeling Drug-Related Deaths in Cook County

This notebook demonstrates interim progress on data acquisition, cleaning, integration, and exploratory analysis. 
It documents the steps taken so far to prepare datasets for spatial and demographic analysis, as outlined in our Project Plan.


In [1]:
# imports

import pandas as pd